In [57]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [58]:
from funcoes import extract_stats

# Tentativa de com base equilibrada p%

In [59]:
p = 30

In [60]:
combinacoes_equilibradas = pd.read_csv("../combinacoes"+str(p)+".txt", header = 0, sep=",")

In [61]:
# le = preprocessing.LabelEncoder()
# nomes = []
# for i in sorted(glob('../letras/caracter*.png')):
#     nomes.append(i.split("/")[1] + "/" + i.split("/")[2])

In [62]:
# le.fit(nomes)

In [63]:
# combinacoes30["char1"] = le.transform(combinacoes30["char1"])
# combinacoes30["char2"] = le.transform(combinacoes30["char2"])

In [64]:
#X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["char1", "char2", "MSE", "ISS", "MSE_centro", "ISS_centro"]],
#                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes_equilibradas[["MSE", "ISS", "MSE_centro", "ISS_centro",\
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",\
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]],
                                                                    combinacoes_equilibradas["resposta"], test_size=0.3, random_state=0)

## SVM

In [66]:
%%time
#from sklearn.svm import SVC
#clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

#SVMparams = { 'kernel' : ('linear', 'poly', 'rbf')} #, 'sigmoid')}

#svr = SVC()
#clf = GridSearchCV(svr, SVMparams)

#clf.fit(X_train, y_train)
#print( "Score: {0}".format(clf.score(X_test, y_test)) )
#print( "Best parameters: " + clf.best_params_ )

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


## Decision Tree

In [67]:
%%time
from sklearn import tree

DTparams = { 'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 3, 7, 40)}

svr = tree.DecisionTreeClassifier()
clf1 = GridSearchCV(svr, DTparams) #criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clf1.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clf1.score(X_test, y_test)) )
print( "Best parameters: " + str(clf1.best_params_) )

Score: 0.7412744877279892
Best parameters: {'criterion': 'entropy', 'min_samples_leaf': 40, 'min_samples_split': 2}
CPU times: user 15.1 s, sys: 0 ns, total: 15.1 s
Wall time: 15.4 s


In [68]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["MSE","ISS",
#                                          "MSE_centro","ISS_centro"],  
#                         class_names = combinacoes30["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

# Random Forest

In [69]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [70]:
%%time

RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.7577122269759063
Best parameters: {'criterion': 'entropy', 'n_estimators': 10, 'min_samples_leaf': 10, 'min_samples_split': 40}
CPU times: user 1min 10s, sys: 0 ns, total: 1min 10s
Wall time: 1min 10s


In [76]:
%%time

from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier(criterion = 'entropy', n_estimators=10, min_samples_leaf=10,
                                       min_samples_split=40)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )

Score: 0.7637919387525333
CPU times: user 576 ms, sys: 0 ns, total: 576 ms
Wall time: 575 ms


## Logistic Regression

In [71]:
%%time
from sklearn.linear_model import LogisticRegression

LRparams = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag')}

svr = LogisticRegression()
clfLR = GridSearchCV(svr, LRparams)
clfLR.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfLR.score(X_test, y_test)) )
print( "Best parameters: " + str(clfLR.best_params_) )

Score: 0.7592884485476245
Best parameters: {'solver': 'newton-cg'}
CPU times: user 11.4 s, sys: 23.2 s, total: 34.6 s
Wall time: 19.1 s


# Multi-layer Perceptron classifier

In [72]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
            'activation' : ('identity', 'logistic', 'tanh', 'relu'),
            'solver' : ('lbfgs', 'sgd', 'adam')}

svr = MLPClassifier()
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.7108759288448547
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 604 ms, sys: 1.74 s, total: 2.34 s
Wall time: 1.25 s


# Binarizando o melhor modelo

In [73]:
#imgA = skio.imread("../letras/caracter1.png")
#imgB = skio.imread("../letras/caracter15.png")

#mse, iss, mse_centro, iss_centro,\
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,\
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro = extract_stats(imgA, imgB)
        
#print(clfRF_balanced.predict( [[mse, iss, mse_centro, iss_centro,
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro]]))

In [77]:
joblib.dump(clfRF_balanced, 'classifier_balanced.pkl') 

['classifier_balanced.pkl']

# Tentativa com base inteira

In [ ]:
combinacoes = pd.read_csv("../combinacoes_2.txt", header = 0)

In [ ]:
# combinacoes["char1"] = le.transform(combinacoes["char1"])
# combinacoes["char2"] = le.transform(combinacoes["char2"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes[["MSE", "ISS", "MSE_centro", "ISS_centro",
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]],
                                                                    combinacoes["resposta"], test_size=0.3, random_state=0)

## SVM

In [ ]:
#%time
#clf.fit(X_train, y_train)
#clf.score(X_test, y_test)

# Decision Tree

In [ ]:
%%time
from sklearn import tree

clftree = tree.DecisionTreeClassifier() # criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clftree.fit( X = X_train, y = y_train )
print( clftree.score(X_test, y_test) )

In [ ]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["char1", "char2","MSE","ISS","MSE_centro","ISS_centro"],  
#                         class_names = combinacoes["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier()

In [ ]:
%%time
clfRF.fit( X = X_train, y = y_train )
print( clfRF.score(X_test, y_test) )

## Binarizando o melhor modelo

In [ ]:
#joblib.dump(clf1, 'classifier.pkl') 

In [ ]:
#joblib.dump(clfRF, 'classifierRF.pkl') 

## testes temporários

In [ ]:
# alguns testes temporários
from funcoes import ler_letras, compare_images
_, letters_dict = ler_letras("../letras.csv")

img1 = skio.imread("../" + 'letras/caracter0.png')
img2 = skio.imread("../" + 'letras/caracter1.png')

m, s = compare_images(img1, img2)
mc, cs = compare_images(img1[10:40,], img2[10:40,])
    
print(clf1.predict([[m, s, mc, cs]])[0] == 1)